In [2]:
import pandas as pd
import numpy as np
import json
import requests

from MusicMetaLinker.linking import linking
from secondhandsongs import Api


Для поиска списка каверов хорошо подходит сайт secondhand. У него есть минус - он не работает с ISCR. В принципе их можно понять, т.к. ISCR появился только в середине 80-х, а многие песни происходят из больших глубин. Поэтому поиск возможен по ID-песни на этом сайте, названию исполнителя и названию песни. Для работы используется API, который работает следующим образом: посылается ссылка-запрос, а сайт(если запрос верный) в ответ присылает json-файл. 
Формат запроса - урл сайта и после знака ? идут опции запроса через &. Например:

'https://secondhandsongs.com/search/performance?title=blackbird&performer=beatles&format=json'

ищется песня(title) с названием blackbird исполнителей beatles, ответ приходит в формате json. Если кликнуть по этой ссылке, то в браузере прям откроется ответ, который мы получаем. Тут есть пример работы с апи, но дано сжато очень и для матерых программистов. https://secondhandsongs.com/page/API
Итак напишем функцию, которая будет делать запрос на предмет наличия какой-либо песни в базе. 

In [39]:
# Функция получает название песни и имя артиста. Можно искать только по одному значению(или песню или артиста)
def SHS_create_url(song = None, artist = None):
    # Первый кусок урла
    api_init = 'https://secondhandsongs.com/search/performance?'
    # строка параметров
    param = ''
    # если передают какую-то информацию в переменной song, то прибавляем это к строке параметров
    if song!=None:
        param += '&title='+song

    # если передают какую-то информацию в переменной artist, то прибавляем это к строке параметров
    if artist!=None:
        param += '&performer='+artist
    # указываем, что нужен ответ в формате json
    param +='&format=json'
    # возвращаем урл как сумму первого куса и куска параметров(
    # [1:] убираем первый &, т.к. первый параметр идет идет без &
    return api_init + param[1:]
    

Для прмера возьмем простой вариант-песня битлов раннего творчества. 

In [44]:
track = 'blackbird'
artist = 'beatles'

In [47]:
shs_url = SHS_create_url(song=track,artist=artist)

# Выполняем GET-запрос 
response = requests.get(shs_url)

# если все хорошо, то бует статус 200
if response.status_code == 200:
    # Получаем данные из ответа (например, JSON-данные)
    json_data = response.json()
    print(json_data)
else:
    # В случае ошибки выводим сообщение
    print(f"Ошибка: {response.status_code}")

{'totalResults': 1, 'resultPage': [{'entityType': 'performance', 'uri': 'https://secondhandsongs.com/performance/1983', 'title': 'Blackbird', 'performer': {'uri': 'https://secondhandsongs.com/artist/41', 'name': 'The Beatles'}, 'isOriginal': True}], 'skippedResults': 0}


Файл json читать глазами не очень удобно. Получим причесанный вид через функцию, которая расставляет переносы строк и пробелы.

In [48]:
formatted_json = json.dumps( json_data, indent=4)

print(formatted_json)

{
    "totalResults": 1,
    "resultPage": [
        {
            "entityType": "performance",
            "uri": "https://secondhandsongs.com/performance/1983",
            "title": "Blackbird",
            "performer": {
                "uri": "https://secondhandsongs.com/artist/41",
                "name": "The Beatles"
            },
            "isOriginal": true
        }
    ],
    "skippedResults": 0
}


 "totalResults": 1 - Один результат поиска. "isOriginal": true - песня оригинал. Кое-что уже имеем. Чтобы получить каверы воспользуемся запросом с указанием ID песни с сайта secondhand. ID в данном случае 1983. В первой ссылке после последнего сплеша написан. Может быть существует путь проще получить ID или обойтись вовсе без него. Я пытался получить через API еще данные с сайта(каверы и т.д.), но не смог понять дальше API, но зато нашел библиотеку для работы с сайтом secondhandsong. Ссылка на гит-хаб https://github.com/arthurtofani/secondhandsongs

 Далее через эту библиотеку получаем подробности о песне. 

In [49]:
# инициализируем класс 
api = Api(api_key=None)
# делаем запрос и получаем ответ
p = api.get_performance(1983)

# выводится полная инфа
print("id:", p.id)
print("title:", p.title)
print("uri:", p.uri)
print("performer_name:", p.performer_name)
print("cover_ids:", p.cover_ids)
print("work_ids:", p.work_ids)
print("release_ids:", p.release_ids)
print("originals:", p.originals)
print("is_original:", p.is_original)

id: 1983
title: Blackbird
uri: https://secondhandsongs.com/performance/1983
performer_name: The Beatles
cover_ids: ['309082', '317181', '318431', '321282', '326238', '326193', '326490', '327649', '329980', '335941', '336150', '338339', '346611', '346681', '352250', '373043', '370724', '7466', '381413', '381509', '381655', '381722', '381750', '284576', '286208', '287312', '287374', '288385', '289079', '296334', '296390', '298375', '301728', '141963', '2000', '1984', '126046', '126513', '2034', '2015', '130223', '1995', '137890', '218681', '138927', '140942', '142908', '143552', '146080', '146160', '250071', '275847', '24812', '269462', '242040', '265362', '278290', '259918', '245757', '2010', '1999', '1998', '2018', '22806', '271752', '2032', '2026', '2014', '1990', '2605', '1997', '33015', '39425', '2056', '1994', '42694', '1996', '2048', '1985', '2028', '2025', '2033', '2051', '2021', '2049', '2030', '2012', '1993', '2007', '2036', '2020', '2039', '1988', '2001', '2005', '1991', '2041

Тут видим уже полную инфу. Нас больше всего интересует cover_ids - список id с каверами песен. Группа известная, поэтому каверов много. 

Момент, который я пока упустил из вида - мы ввели название группы beatles, но на самом деле мы его не знали. В исходном датасете есть только название песни. Попробуем получить название группы через библиотеку, которую яндекс рекомендовал. Отдельной песни заслуживает установка этой библиотеки. Эта библиотека ищет информацию на нескольких сайтах и непонятным образом суммирует её. Исходными данными выступят название песни и ISRC.  

Исходныеми данными будут ISRC. Делаем запрос, чтобы получить подробную информацию о песне, собрать больше информации. 

In [23]:
#song = 'я сошла с ума'
in_isrc = "RUA110100031"

In [51]:
linker = linking.Align( isrc=in_isrc     )
track_name = linker.get_track()
artist_name = linker.get_artist()
release_year = linker.get_release_date()

print(track_name, artist_name, release_year)

Searching for None - None on YouTube Music
Ya Soshla S Uma None 2006


Непонятный вывод - почему нет имени артиста и год указан как 2006? Ведь песня известная Тату и год 2000 должен быть. Попробуем отдельно к каждой базе обратиться. 

In [56]:
# обращаемся к musicbrain
musicbrain = linking.MusicBrainzAlign(isrc = in_isrc)
#очень длинный вывод в json
result = musicbrain._search_by_isrc()
result = json.dumps( result, indent=4)
print(result)



{
    "isrc": {
        "id": "RUA110100031",
        "recording-list": [
            {
                "id": "f08b7115-a967-4c0a-97dd-7badfed39173",
                "title": "Ya Sholsa S Uma",
                "length": "214546",
                "artist-credit": [
                    {
                        "artist": {
                            "id": "7c20f558-d664-42bd-b6f1-81acf9dbb72d",
                            "type": "Group",
                            "name": "t.A.T.u.",
                            "sort-name": "t.A.T.u.",
                            "disambiguation": "Russian duo"
                        }
                    }
                ],
                "release-list": [
                    {
                        "id": "3ec33ed3-8dae-4674-bf53-b8dce4bdb8c6",
                        "title": "\u042f \u0441\u043e\u0448\u043b\u0430 \u0441 \u0443\u043c\u0430",
                        "status": "Official",
                        "quality": "normal",
             